# Capítulo 07 — Instruction Tuning

Este notebook acompanha o Capítulo 07 da série **Fazendo um LLM do Zero**.

🎯 **Objetivos deste notebook:**
- Interpretar instruções estruturadas
- Implementar Mascaramento de Loss para focar na resposta
- Transformar um modelo base em um assistente conversacional


## 1. Setup e Configuração

In [ ]:
import os
import sys

REPO_URL = "https://github.com/vongrossi/fazendo-um-llm-do-zero.git"
REPO_DIR = "fazendo-um-llm-do-zero"

if not os.path.exists(REPO_DIR):
    !git clone {REPO_URL}

os.chdir(REPO_DIR)
sys.path.append(os.getcwd())
print("Diretório atual:", os.getcwd())


In [ ]:
!pip -q install -r 07-instruction-tuning/requirements.txt

import torch
import torch.nn.functional as F
import torch.nn as nn
import random
import numpy as np
from lib.gptmini import GPTConfig, GPTMini

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)


## 2. Dataset e Vocabulário

Diferente dos capítulos anteriores, agora trabalhamos com exemplos de **Comando** e **Resposta**. Definimos o vocabulário antes de carregar o modelo.

In [ ]:
dataset = [
    {"instruction": "Explique Machine Learning", "response": "É uma área da IA que permite aprender padrões a partir de dados."},
    {"instruction": "Traduza: bom dia", "response": "Good morning"},
    {"instruction": "O que é um token?", "response": "É a unidade básica de texto processada pelo modelo."},
    {"instruction": "Resuma: GPTs são Transformers", "response": "Modelos GPT usam arquitetura Transformer para linguagem."}
]

def format_instruction(item):
    return f"### Instrução:\n{item['instruction']}\n\n### Resposta:\n{item['response']}"

all_text = "".join([format_instruction(d) for d in dataset])
chars = sorted(set(all_text))
stoi = {c:i for i,c in enumerate(chars)}
itos = {i:c for c,i in stoi.items()}
vocab_size = len(chars)

def encode(text): return [stoi[c] for c in text if c in stoi]
def decode(tokens): return "".join([itos[t] for t in tokens])

print(f"Vocabulário carregado: {vocab_size} caracteres")


## 3. Inicialização e Carregamento de Pesos

Vamos carregar o modelo pré-treinado no Capítulo 05 para servir de cérebro base.

In [ ]:
config = GPTConfig(
    vocab_size=vocab_size, 
    context_size=64,
    d_model=128,
    n_heads=4,
    n_layers=2
)
backbone = GPTMini(config).to(device)

checkpoint_path = "gpt_checkpoint.pt"
try:
    p = checkpoint_path if os.path.exists(checkpoint_path) else "05-pre-treinamento/" + checkpoint_path
    ckpt = torch.load(p, map_location=device)
    state_dict = ckpt["state_dict"] if "state_dict" in ckpt else ckpt
    model_dict = backbone.state_dict()
    # Filtra apenas o que é compatível
    pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
    backbone.load_state_dict(pretrained_dict, strict=False)
    print(f"✅ {len(pretrained_dict)} camadas do Capítulo 05 carregadas!")
except:
    print("⚠️ Checkpoint não encontrado — iniciando modelo do zero")


## 4. Pipeline de Treinamento SFT

Preparamos o dataset com a **Máscara de Loss** para que o modelo aprenda apenas com a resposta.

In [ ]:
def build_instruction_dataset(data, context_size=64):
    X, Y, mask = [], [], []
    for item in data:
        input_text = f"### Instrução:\n{item['instruction']}\n\n### Resposta:\n"
        full = input_text + item["response"]
        instruction_tokens = encode(input_text)
        full_tokens = encode(full)

        for i in range(len(full_tokens)-context_size):
            x = full_tokens[i:i+context_size]
            y = full_tokens[i+1:i+context_size+1]
            m = [0]*len(instruction_tokens) + [1]*(context_size-len(instruction_tokens))
            X.append(x); Y.append(y); mask.append(m[:context_size])

    return torch.tensor(X).to(device), torch.tensor(Y).to(device), torch.tensor(mask).to(device)

X, Y, MASK = build_instruction_dataset(dataset)

class InstructionGPT(nn.Module):
    def __init__(self, backbone): 
        super().__init__(); self.backbone = backbone
    def forward(self, x, y=None, mask=None):
        logits, _ = self.backbone(x)
        loss = None
        if y is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), y.view(-1), reduction="none")
            loss = (loss * mask.view(-1)).mean()
        return logits, loss

model = InstructionGPT(backbone).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)


In [ ]:
print("🚀 Treinando Alinhamento...")
for step in range(301):
    idx = torch.randint(0, X.size(0), (8,))
    logits, loss = model(X[idx], Y[idx], MASK[idx])
    optimizer.zero_grad(); loss.backward(); optimizer.step()
    if step % 100 == 0: print(f"Step {step} | Loss {loss.item():.4f}")


## 5. Testando o Assistente

Agora o modelo deve ser capaz de responder comandos.

In [ ]:
@torch.no_grad()
def generate(model, start, max_tokens=80):
    tokens = encode(start)
    tokens = torch.tensor(tokens).unsqueeze(0).to(device)
    for _ in range(max_tokens):
        logits, _ = model(tokens)
        next_token = torch.argmax(logits[:, -1, :], dim=-1)
        tokens = torch.cat([tokens, next_token.unsqueeze(1)], dim=1)
    return decode(tokens.squeeze().tolist())

print(generate(model, "### Instrução:\nExplique Machine Learning\n\n### Resposta:\n"))


In [ ]:
torch.save(model.state_dict(), "07_instruction_gpt.pt")
print("✅ Checkpoint final salvo!")


## 6. Conclusão

Você completou a série! 

Transformou um modelo que apenas previa o próximo token em um assistente capaz de seguir intenções humanas. 

Este é o fundamento do que faz o ChatGPT, o Claude e o Gemini serem tão úteis no nosso dia a dia.